In [1]:
import numpy as np

from utils import timestamp, YEAR
from utils.arrayview import ArrayView, TimeseriesView

from prediction.models.fit_model import TSModel
from prediction.models.model_parameters import ModelParameters
from prediction.models.factor_management import FactorList
from prediction.models.preprocessing import load_slices, print_factor_order

from prediction.models.prediction import factornames_trimmed
from prediction.models.parameters import factor_build_end

In [2]:
av = ArrayView.from_file('../datadev/brain_final2cut.av.bcolz')

In [3]:
av_new = ArrayView.from_file('../datadev/roll_normspeed.av.bcolz')

In [4]:
av.norm_speed = av_new.prediction

In [5]:
factornames_trimmed += ['norm_speed']

In [6]:
tsav = load_slices(path='../datadev/')

In [7]:
pars = ModelParameters(av, oos_start=factor_build_end+YEAR, depth=3, lmbd=10, verbose=True)

In [8]:
fl = FactorList(av, factornames_trimmed)

In [9]:
fl.preprocess(pars)
factors = fl.asmatrix()

INFO:models:Getting factors from av and rescaling...


. . . . .

INFO:models:Filling in missing values...
INFO:models:Computing each factor as linear combination of all the others...


 . . . . .

INFO:models:Number of missing patterns: 8970


 . . . . . . . .


In [10]:
tsmod = TSModel(factors, tsav, pars)

In [11]:
tsmod.fit_slices()

INFO:models:Fitting slice 0
prediction/models/fit_model.py:153: RuntimeWarning: divide by zero encountered in log
  stats, self.step2probs[sl] = self.concat_and_fit(strata, result, nonrunner, [fback, flay, np.log(probs)], ts_idx, valid2, verbose=False, step=2)
INFO:models:Fitting slice 1
INFO:models:Fitting slice 2
INFO:models:Fitting slice 3
INFO:models:Fitting slice 4
INFO:models:Fitting slice 5
INFO:models:Fitting slice 6
INFO:models:Fitting slice 7
INFO:models:Fitting slice 8
INFO:models:Fitting slice 9


In [12]:
tsmod.stats1.ll

array([[-1829.61768357, -1829.61768357, -2084.67887864],
       [-1835.13413214, -1835.13413214, -2072.30436513],
       [-1840.33763733, -1840.33763733, -2057.89829756],
       [-1844.09409047, -1844.09409047, -2038.27775848],
       [-1861.57429242, -1861.57429242, -1991.21234572],
       [-1879.34836153, -1879.34836153, -1965.35947037],
       [-1915.45955141, -1915.45955141, -1918.50530061],
       [-1929.85159447, -1929.85159447, -1894.15716885],
       [-1932.34662518, -1932.34662518, -1891.3556987 ],
       [-1936.40109809, -1936.40109809, -1878.82143271],
       [           nan,            nan,            nan]])

In [13]:
tsmod.stats2.ll

array([[-1818.62669876, -2102.5762705 , -2102.5762705 ],
       [-1824.16254691, -2089.862647  , -2089.862647  ],
       [-1829.48060546, -2074.19567574, -2074.19567574],
       [-1832.92080827, -2052.85636579, -2052.85636579],
       [-1850.81062904, -1998.76557221, -1998.76557221],
       [-1868.77958549, -1965.32708145, -1965.32708145],
       [-1904.50948427, -1911.32532067, -1911.32532067],
       [-1919.50365606, -1878.36099874, -1878.36099874],
       [-1922.24026342, -1875.31672739, -1875.31672739],
       [-1926.77868628, -1862.7657947 , -1862.7657947 ],
       [           nan,            nan,            nan]])

In [14]:
print_factor_order(tsmod.stats1, factornames_trimmed)

  0:                                                              z027f9f0f5    t-score sum: 92.05
  1:                                                              zec0c22a48    t-score sum: 86.02
  2:                                                              z412893062    t-score sum: 64.71
  3:                                                              ze4c91eac0    t-score sum: 61.41
  4:                                                              z5981b9f89    t-score sum: 56.32
  5:                                                              z77c9cc0a5    t-score sum: 54.45
  6:                                                              z245159235    t-score sum: 49.87
  7:                                                              z34b78e584    t-score sum: 46.87
  8:                                                              zb5dc14260    t-score sum: 46.79
  9:                                                              z6809c316d    t-score sum: 42.84
 10:      

In [15]:
tsmod.write_simdata('simdata.p')